In [1]:
import numpy as np    
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#input images (55000), each of dimension 28*28 with a single channel
#one-hot encoded labels
mnist.train.images.shape,mnist.train.labels.shape

((55000, 784), (55000, 10))

## Defining weights and biases and other parameters

In [4]:
#image info
n_height = 28
n_width = 28
channels = 1
pixels = n_height*n_width

#convolution layers
units_conv1 = 32
units_conv2 = 64
#units_conv3 = 64

#filter dimension
kernel_c1 = 5
kernel_c2 = 3
#kernel_c3 = 3

#strides of convolution layers
stride_c1 = 1
strides_c2 = 1

#pooling layers
kernel_m1 = 2
kernel_m2 = 2

n_output = 10
n_hidden = 1024

#in our case (7,7,64)
hidden_input = (n_width // (kernel_m1*kernel_m2)) * (n_height // (kernel_m1*kernel_m2)) * units_conv2 

W = {
    
    'WC1' : tf.Variable(tf.random_normal([kernel_c1,kernel_c1,channels,units_conv1])),
    'WC2' : tf.Variable(tf.random_normal([kernel_c2,kernel_c2,units_conv1,units_conv2])),
    'WH' : tf.Variable(tf.random_normal([hidden_input,n_hidden])),
    'WO' : tf.Variable(tf.random_normal([n_hidden,n_output]))
}

B = {
    'BC1' : tf.Variable(tf.random_normal([units_conv1])),
    'BC2' : tf.Variable(tf.random_normal([units_conv2])),
    'BH' : tf.Variable(tf.random_normal([n_hidden])),
    'BO' : tf.Variable(tf.random_normal([n_output])),
}

Instructions for updating:
Colocations handled automatically by placer.


### Convolution layer

In [5]:
def conv(x,W,B,strides = 1):
    c = tf.nn.conv2d(x,W,padding='SAME',strides=[1,strides,strides,1])
    c = tf.nn.bias_add(c,B)
    c = tf.nn.relu(c)
    return c

### Max pool layer

In [6]:
def maxPool(x,kernel = 2,strides = 2):
    pool = tf.nn.max_pool(x,ksize=[1,kernel,kernel,1],strides=[1,strides,strides,1],padding='VALID')
    return pool

## Forward Propogation

### Our Model:
    - conv layer with 32 units
    - max pool with kernel = 2
    - conv layer with 64 units
    - max pool with kernel = 2
    - hidden layer with 1024 units
    - output layer with 10 units(for 1-10 output)

In [7]:
def forward(x,W,B):
    x = tf.reshape(x,[-1,n_width,n_height,channels])
    c1 = conv(x,W['WC1'],B['BC1'],stride_c1)
    c1_pool = maxPool(c1,kernel_m1)
    
    c2 = conv(c1_pool,W['WC2'],B['BC2'],strides_c2)
    c2_pool = maxPool(c2,kernel_m2)
    
    hidden_in = tf.reshape(c2_pool,[-1,hidden_input])
    z_hidden = tf.add(tf.matmul(hidden_in,W['WH']),B['BH'])
    hidden_out = tf.nn.relu(z_hidden)
    
    output = tf.add(tf.matmul(hidden_out,W['WO']),B['BO'])
    
    return output

In [8]:
x = tf.placeholder("float", [None, pixels])
y = tf.placeholder(tf.int32, [None, n_output])
pred = forward(x, W, B)

In [9]:
#cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))


In [10]:
tf.trainable_variables()

[<tf.Variable 'Variable:0' shape=(5, 5, 1, 32) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 <tf.Variable 'Variable_2:0' shape=(3136, 1024) dtype=float32_ref>,
 <tf.Variable 'Variable_3:0' shape=(1024, 10) dtype=float32_ref>,
 <tf.Variable 'Variable_4:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'Variable_5:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'Variable_6:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'Variable_7:0' shape=(10,) dtype=float32_ref>]

In [11]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.02)
optimize = optimizer.minimize(cost)

In [12]:
sess = tf.Session()
#sess = tf.Session(config=tf.ConfigProto(device_count={'GPU': 0}))
sess.run(tf.global_variables_initializer())
    

In [13]:
#training
batch_size = 64
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_x , y:batch_y})
        total_cost += c
    print(total_cost)

1030057.2806085367
16208.970570193786
11091.667490085478
8600.54240705022
7623.44033175798
5661.429063851742
4483.154138681268
3518.7280014080143
2437.8584572807063
2090.4111654885523
2475.2364497979834
865.0068956241296
1395.403031199794
884.3172889280404
590.4094723644994
627.8155071126237
552.8733425714797
311.3910928495265
218.9455807376465
226.1190361275269
279.706555410291
185.73051727114887
194.1771504254575
202.0022052564309
163.8000511422615


In [14]:
pred.shape

TensorShape([Dimension(None), Dimension(10)])

In [15]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_predictions  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                              y:mnist.test.labels})
correct_predictions.sum()

9617

## Accuracy

In [23]:
acc = correct_predictions.sum()/len(mnist.test.labels)
print("Accuracy %.2f" %acc)

Accuracy 0.96
